In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163750 sha256=a9951e1e64c7ad63a0fdcaf691f4a8244f16b240bb97367485978c9c5ed56ed2
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import surprise
import os

In [3]:
os.chdir("/content/drive/MyDrive/Cases/filmtrust")
ratings = pd.read_csv("ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

uid  iid  rating
0    1    1     2.0
1    1    2     4.0
2    1    3     3.5
3    1    4     3.0
4    1    5     4.0

In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


In [5]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [7]:
similarity_options = {'name': 'cosine', 'user_based': True}
# Default k = 40
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


Expected rating for user 1 for item 217:

In [11]:
pred = algo.predict(uid='1',iid='217')
print(pred.est)

3.0028030537791928


In [12]:
pred

Prediction(uid='1', iid='217', r_ui=None, est=3.0028030537791928, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

Total Items:

In [13]:
iids = ratings['iid'].unique()
print(iids)

[   1    2    3 ... 2069 2070 2071]


The list of items rated by user 1:

In [14]:
u_iid = ratings[ratings['uid']==1]['iid'].unique()
print(u_iid)

[ 1  2  3  4  5  6  7  8  9 10 11 12]


In [15]:
len(u_iid)

12

List of the items not rated by user 1:

In [17]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[  13   14   15 ... 2069 2070 2071]


In [18]:
len(iids_to_predict)

2059

Extracting the estimated rating from iids_to_predict

In [ ]:
testset = [[1,iid,0.] for iid in iids_to_predict]
testset

In [20]:

predictions = algo.test(testset)

In [21]:
type(predictions), len(predictions)

(list, 2059)

In [22]:
predictions[:5]

[Prediction(uid=1, iid=13, r_ui=0.0, est=3.7251325884696556, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=14, r_ui=0.0, est=1.0415223651355818, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid=1, iid=15, r_ui=0.0, est=2.331059882750255, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=1, iid=16, r_ui=0.0, est=3.350290135026733, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=1, iid=17, r_ui=0.0, est=3.1123092893419364, details={'actual_k': 40, 'was_impossible': False})]

Getting the item with highest expected rating

In [24]:
pred_ratings = [ (predictions[i].iid,predictions[i].est) for i in range(0,len(predictions)) ]
predicted_rating = pd.DataFrame(pred_ratings, columns=['iid','est_rating'])
predicted_rating = pd.DataFrame(pred_ratings, columns=['iid','est_rating'])
predicted_rating.sort_values(by='est_rating', ascending=False)

iid  est_rating
1255  1268         4.0
1630  1643         4.0
1654  1667         4.0
149    162         4.0
1653  1666         4.0
...    ...         ...
861    874         0.5
1772  1785         0.5
391    404         0.5
1541  1554         0.5
1985  1998         0.5

[2059 rows x 2 columns]

Tuning for best K

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

### User-Based Filtering

In [ ]:
param_grid = {'k': np.arange(30,70,10),  'user_based':[True]}
param_grid

{'k': array([30, 40, 50, 60]), 'user_based': [True]}

In [ ]:
kfold = KFold(n_splits=5, random_state=23, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [ ]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.8641633357915124


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'k': 60, 'user_based': True}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


The recommendations can be generated for any user with the object **algo**.

### Item-Based Filtering

In [ ]:
param_grid = {'k': np.arange(30,70,10), 'user_based':[False]}
param_grid

{'k': array([30, 40, 50, 60]), 'user_based': [False]}

In [ ]:
kfold = KFold(n_splits=5, random_state=23, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [ ]:
gs.fit(data)

Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.8641633357915124


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'k': 60, 'user_based': False}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


The recommendations can be generated for any user with the object **algo**.